# Implementation der Spiellogik

Im Folgenden ist das Spiel Othello implementiert. Ausgangspunkt für diese Implementierung ist Das Python Gui für Othello von Kevan Nguyen, welches unter <https://github.com/kevannguyen/Othello> verfügbar ist.

## Importieren der externen Abhängigkeiten

Die Implementation stützt sich für bessere Performanz auf die Python, Bibliothek `numpy` welche unter anderem homogene Felder und Matrizen implementiert.

In [ ]:
import numpy

## Globale Konstanten

`BOARD_SIZE` gibt die Anzahl an Zeilen und Spalten des quadratischen Spielfelds an.

Die Konstanten `BLACK`, `WHITE` und `NONE` werden auf die Zahlenwerte -1 , 1 und 0 abgebildet und werden für mehrere Zwecke genutzt:
- zur Repräsentation des Spielbretts
- zur Identifikation eines Spielers
- zur Berechnung der Heuristiken.

In [ ]:
BOARD_SIZE = 8

BLACK = -1 # MINIMIZING PLAYER
WHITE = 1 # MAXIMIZING PLAYER
NONE = 0

## Game State

Die Klasse GameState repäsentiert den Spielzustand. Dieser besteht aus dem Spielfeld `board`, welches durch einen Numpy Array repräsentiert wird, sowie dem Spieler `turn`, der am Zug ist. Zusätzlich werden weitere Informationen wie, mögliche Züge, Frontier und Anzahl an Spielsteinen gehalten, um zu vermeiden, dass diese häufig neu berechnet werden müssen.

In [ ]:
class GameState:
    def __init__(self):
        self.board = numpy.zeros((BOARD_SIZE, BOARD_SIZE), dtype=numpy.int8)
        self.board[3, 3] = WHITE
        self.board[3, 4] = BLACK
        self.board[4, 3] = BLACK
        self.board[4, 4] = WHITE
        self.frontier = {(2,2),(2,3),(2,4),(2,5),
                         (3,2),(3,5),(4,2),(4,5),
                         (5,2),(5,3),(5,4),(5,5)}
        self.turn = BLACK
        self.num_pieces = 4
        self.game_over = False
        self.possible_moves = [(2, 3), (3, 2), (4, 5), (5, 4)]
    def __lt__(self, other):
        return True

In [ ]:
directions = {(-1,-1),(0,-1),(1,-1),(-1,0),(1,0),(-1,1),(0,1),(1,1)}

Erhält die Funktion `make_move` ein ungültigen Spielzug, so wird eine `InvalidMoveException` ausgelöst.

In [ ]:
class InvalidMoveException(Exception):
    pass

`make_move` führt im Spielzustand `state` falls möglich einen Zug auf die Koordinaten `row` und `col` aus.

In [ ]:
def make_move(state, row, col):
    if (row, col) not in state.frontier:
        print("(", row, ", ", col, ") not in Frontier")
        raise InvalidMoveException
    next_turn = state.turn
        
    possible_directions = adjacent_opposite_color_directions(state, row, col, state.turn)
    for (row_dir, col_dir) in possible_directions:
        if is_valid_directional_move(state, row, col, row_dir, col_dir, state.turn):
            next_turn = -state.turn
            convert_adjacent_cells_in_direction(state, row, col, row_dir, col_dir, state.turn)

    if next_turn != state.turn:
        state.num_pieces += 1
        state.board[(row, col)] = state.turn
        update_frontier(state, row, col)
        if can_move(state, next_turn):
            state.turn = next_turn
        elif not can_move(state, state.turn):
                state.game_over = True
                return state
        state.possible_moves = get_possible_moves(state, state.turn)
    else:
        raise InvalidMoveException()
    return state

Diese Funktion überprüft für einen gegebenen Zustand `state` ob ein Zug auf die angegebenen Koordinaten `row` und `col` für den Spieler `turn` möglich ist. Das Ergebnis wird als Wahrheitswert zurückgegeben.

In [ ]:
def is_move_valid(state, row, col, turn):
    dirs = [(-1,-1),(0,-1),(1,-1),(-1,0),(1,0),(-1,1),(0,1),(1,1)]
    for rowdelta, coldelta in dirs:
        try:
            if state.board[row + rowdelta, col + coldelta] == -turn and is_valid_directional_move(state, row, col, rowdelta, coldelta, turn):
                return True
        except IndexError:
            continue
    return False

`get_winner` bestimmt für einen Endzustand den Gewinner des Spiels. Gewinnt Weiß, so wird der Wert 1 zurückgegeben. Gewinnt Schwarz, wird der Wert -1 zurückgegeben. Bei einem Unentschieden wird der Wert 0 zurückgegeben.

In [ ]:
def get_winner(state):
    black_disks = count_disks(state, BLACK)
    white_disks = count_disks(state, WHITE)
    if black_disks > white_disks:
        return BLACK
    if white_disks > black_disks:
        return WHITE
    else:
        return NONE

Die Funktion `get_possible_moves` bestimmt für einen Spielzustand `state` und den Spieler `player` die Möglichen Züge, die der Spieler machen kann. Die Züge werden als Liste von Koordinaten zurückgegeben.

In [ ]:
def get_possible_moves(state, player):
    possible_moves = []
    for (row, col) in state.frontier:
            if is_move_valid(state, row, col, player):
                possible_moves.append((row, col))
    return possible_moves

`can_move` überprüft ob der Spieler `player` im Spielzustand `state` einen Zug machen kann und gibt das Resultat als Wahrheitswert zurück.

In [ ]:
def can_move(state, player):
    return len(get_possible_moves(state, player)) != 0

`adjacent_opposite_color_directions` betrachtet die 8 an die gegebenen Koordinaten angrenzenden Felder. Alle Felder die einen für den Spieler `turn` gegnerischen Stein enthalten werden als Liste von Koordinaten zurückgegeben.

In [ ]:
def adjacent_opposite_color_directions(state, row, col, turn):
    dir_list = []
    for rowdelta in range(-1, 2):
        if not 0 <= row+rowdelta < 8:
            continue
        for coldelta in range(-1, 2):
            if not 0 <= col+coldelta < 8:
                continue
            if state.board[row + rowdelta, col + coldelta] == -turn:
                dir_list.append((rowdelta, coldelta))
    return dir_list

Überprüft ob beim setzen eines Steins des Spielers `turn` in das durch `row` und `col` gegebene Feld in die durch `rowdelta` und `coldelta` gegebene Richtung, nach den Regeln des Spiels Othello gegnerische Steine umgedreht werden Können.

In [ ]:
def is_valid_directional_move(state, row, col, rowdelta, coldelta, player):
        current_row = row + rowdelta
        current_col = col + coldelta

        last_cell_color = -player

        while True:
            if not (0 <= current_row < 8 and 0 <= current_col < 8):
                break
            if state.board[current_row, current_col] == NONE:
                break           
            if state.board[current_row, current_col] == player:
                last_cell_color = player
                break

            current_row += rowdelta
            current_col += coldelta
            
        return last_cell_color == player

Diese Funktion dreht im Spielzustand `state`, ausgehend von dem durch `row` und `col` gegebenen Feld, die für den Spieler `player` generischen Steine in die durch `rowdelta` und `coldelta` gegebene Richtung um.

In [ ]:
def convert_adjacent_cells_in_direction(state, row, col, rowdelta, coldelta, player):
    ''' If it can, converts all the adjacent/contiguous cells on a turn in
        a given direction until it finally reaches the specified cell's original color '''
    if is_valid_directional_move(state, row, col, rowdelta, coldelta, player):
        current_row = row + rowdelta
        current_col = col + coldelta
        
        while state.board[current_row, current_col] == -player:
            state.board[(current_row, current_col)] = player
            current_row += rowdelta
            current_col += coldelta

`update_frontier` wird nach jedem Zug aufgerufen um die Menge Frontier zu aktualisieren. Die durch `row` und `col` Gegebene Koordinate wird entfernt, während die Koordinaten von leeren umliegenden Feldern hinzugefügt werden.

In [ ]:
def update_frontier(state, row, col):
        for current_row in range(row-1, row+2):
            if not 0 <= current_row < 8:
                continue
            for current_col in range(col-1, col+2):
                if not 0 <= current_col < 8:
                    continue
                if state.board[current_row, current_col] == NONE:
                    state.frontier.add((current_row, current_col))
        state.frontier.remove((row, col))

Die Funktion `count_disks` zählt die Steine, die der Spieler `player` im Spielzustand `state` auf dem Spielfeld hat

In [ ]:
def count_disks(state, player):
    return numpy.count_nonzero(state.board == player)

`get_player_string` konvertiert die ID des Spielers `player` in dessen Name. Ist `player == NONE` so wird 'Nobody' zurückgegeben.

In [ ]:
def get_player_string(player):
    return {BLACK: 'Black', WHITE: 'White', NONE: 'Nobody'}[player]

# Implementation der Grafischen Benutzeroberfläche

Im folgenden Abschnitt wird eine Benutzeroberfläche für das Spiel Othello implementiert.

## Importieren der externen Abhängigkeiten

Die Grafische Benutzeroberfläche verwendet zur Darstellung des Spielzustandes, zum Anzeigen weiterer Informationen sowie für die Benutzerinteraktion die Bibliotheken `ipycanvas` und `ipywidgets`. Diese Lassen sich direkt im Jupyter Notebook verwenden.

Zusätzlich werden aus dem Paket `math` der Python Standardbibliothek die Variable `pi` sowie die Funktion `floor` benötigt.

In [ ]:
import ipycanvas
import ipywidgets
import math

## Globale Konstanten

## Canvas Initialisieren

`SHOW_FRONTIER` gibt an ob in der Visualisierung leere Felder, die an bereits gesetzte Spielsteine angrenzen hervorgehoben werden sollen.


`SHOW_POSSIBLE_MOVES` gibt ob für den aktuell ziehenden Spieler mögliche Züge visualisiert werden sollen.

In [ ]:
SHOW_FRONTIER = False
SHOW_POSSIBLE_MOVES = True

In [ ]:
CELL_SIZE = 60

CANVAS_SIZE = BOARD_SIZE * CELL_SIZE

canvas = ipycanvas.MultiCanvas(2, width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas[0].fill_style = 'darkgreen'
canvas[0].stroke_style = 'black'
canvas[0].fill_rect(0, 0, CANVAS_SIZE, CANVAS_SIZE)
canvas[0].begin_path()
for i in range(BOARD_SIZE+1):
    pos = i * CELL_SIZE
    canvas[0].move_to(pos, 0)
    canvas[0].line_to(pos, CANVAS_SIZE)
    canvas[0].move_to(0, pos)
    canvas[0].line_to(CANVAS_SIZE, pos)
canvas[0].stroke()

## Widgets Initialisieren

Das `score_lbl` Widget enthält die Steinzahl beider Spieler im aktuellen Spielzustand

In [ ]:
score_lbl = ipywidgets.widgets.Label()

Das `turn_lbl` Widget nennt den Spieler, der gerade am Zug ist

In [ ]:
turn_lbl = ipywidgets.widgets.Label()

Das `output` Widget macht die Ausgabe mithilfe von `print()`, sowie die Ausgabe von Fehlermeldungen trotz der Verwendung von IPyWidgets und IPyCanvas möglich.

In [ ]:
output = ipywidgets.widgets.Output()

Die Funktion `display_board` stellt den angegebenen Spielzustand dar, indem zunächst der Canvas aktualisiert, und dann zusammen mit den Status-Widgets angezeigt wird.

In [ ]:
def display_board(state):
    update_output(state)
    display(canvas)
    display(score_lbl)
    display(turn_lbl)
    display(output)

In der Funktion `update_output` wird der Spielzustand `state` auf den Canvas gezeichnet.

In [ ]:
def update_output(state):
    with ipycanvas.hold_canvas(canvas):
        canvas[1].clear()
        for ((x, y), val) in numpy.ndenumerate(state.board):
            if val == NONE:
                continue
            elif val == BLACK:
                canvas[1].fill_style = 'black'
            else:
                canvas[1].fill_style = 'white'
            canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 2.2, 0, 2 * math.pi)
            
        if SHOW_FRONTIER:
            for (x, y) in state.frontier:
                canvas[1].fill_style = 'gray'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)
        
        if SHOW_POSSIBLE_MOVES:
            for (x, y) in get_possible_moves(state, state.turn):
                if state.turn == BLACK:
                    canvas[1].fill_style = 'black'
                else:
                    canvas[1].fill_style = 'white'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)
            
    score_lbl.value = f'Black Player : {count_disks(state, BLACK)} White Player : {count_disks(state, WHITE)}'
    if state.game_over:
        turn_lbl.value = f'{get_player_string(get_winner(state))} wins'
    else:
        turn_lbl.value = f'{get_player_string(state.turn)}s Move'

Für den menschlichen Spieler ist es nötig festzustellen, ob dieser auf das Spielfeld geklickt hat, dies geschieht in der callback funktion `mouse_down` welche die x und y Koordinaten des Mausklicks relativ zum Canvas erhält. Auf Basis dieser Position wird, falls möglich, ein Zug auf das angeklickte Feld gemacht. Die Funktion wird durch den aufruf von `Canvas.on_mouse_down` bei IPyCanvas als Callback Funktion registriert.

In [ ]:
def mouse_down(x_px, y_px):
    if not state.game_over:
        with output:
            x = math.floor(x_px / CELL_SIZE)
            y = math.floor(y_px / CELL_SIZE)
            try:
                make_move(state, x, y)
            except InvalidMoveException:
                print('Invalid Move')
            update_output(state)
            next_move(state)

canvas[1].on_mouse_down(mouse_down)

In [ ]:
#state = GameState()

In [ ]:
#display_board(state)